<a href="https://colab.research.google.com/github/blue0620/NDLkotenOCR-GoogleColabVersion/blob/main/NDLOCR_googlecolabversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. GPUの情報を確認する

In [ ]:
!nvidia-smi

# 1. NDLOCRのリポジトリをcloneする(--recursiveを忘れずに！)

In [ ]:
!git clone --recursive https://github.com/ndl-lab/ndlocr_cli

# 2. 必要なパッケージをインストールする

In [ ]:
PROJECT_DIR="/content/ndlocr_cli"

In [ ]:
!pip install -r {PROJECT_DIR}/requirements.txt
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install mmcv-full==1.4.0 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.8.0/index.html
##numpyのバージョン問題でcolabでは動かなかったのでアップデートする(参考:https://stackoverflow.com/questions/66060487/valueerror-numpy-ndarray-size-changed-may-indicate-binary-incompatibility-exp)
#!pip install --upgrade numpy

In [ ]:
%cd {PROJECT_DIR}/src/ndl_layout/mmdetection
!python setup.py bdist_wheel
!pip install dist/*.whl
%cd /content

# 4. OCRに必要な学習済みモデルをダウンロードする

In [ ]:
%cd {PROJECT_DIR}
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/mojilist_NDL.txt -P ./src/text_recognition/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/ndlenfixed64-mj0-synth1.pth -P ./src/text_recognition/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/ndl_layout_config.py -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/epoch_140_all_eql_bt.pth -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/separate_pages_ssd/weights.hdf5 -P ./src/separate_pages_ssd/ssd_tools
%cd /content/

# 5. 環境変数を追加する

In [ ]:
import os
os.environ["PYTHONPATH"]=os.environ["PYTHONPATH"]+":"+f"{PROJECT_DIR}/src/text_recognition/deep-text-recognition-benchmark"

# 6. PDFを画像に変換するためのパッケージのインストール

In [ ]:
!apt-get install poppler-utils
!pip install pdf2image 

# 7. テキスト化したいPDFをダウンロードする
今回は、ROIS-DS人文学オープンデータ共同利用センター(http://codh.rois.ac.jp/
)が提供している

近代雑誌データセット　http://codh.rois.ac.jp/modern-magazine/
から、

東洋学芸雑誌(https://dglb01.ninjal.ac.jp/ninjaldl/bunken.php?title=toyogakuge)

第一号(https://dglb01.ninjal.ac.jp/ninjaldl/toyogakuge/001/PDF/tygz-001.pdf)

をダウンロードしてみます。

In [ ]:
!curl https://dglb01.ninjal.ac.jp/ninjaldl/toyogakuge/001/PDF/tygz-001.pdf -o /content/tygz-001.pdf

# 8. PDFをjpeg画像に変換する

In [ ]:
from pathlib import Path
from pdf2image import convert_from_path
import os
pdf_path = Path("/content/tygz-001.pdf")
os.makedirs("/content/tygz-001/img",exist_ok=True)
img_path=Path("/content/tygz-001/img")

convert_from_path(pdf_path, output_folder=img_path,fmt='jpeg',output_file=pdf_path.stem,dpi=100)

# 9. OCRの実行

/content/tygz-001以下のimgディレクトリ内の画像を処理し、
/content/tygz-001_outputに出力する場合

In [ ]:
%cd {PROJECT_DIR}
!python main.py infer /content/tygz-001 /content/tygz-001_output -s s -x

# 10. 結果の確認

In [ ]:
import glob
import os
for fpath in sorted(glob.glob("/content/tygz-001_output/tygz-001/txt/*_main.txt")):
    with open(fpath) as f:
        txtdata=f.read()
        print(os.path.basename(fpath).replace("_main.txt",""))
        print(txtdata)